# Normalizing institution `orgname` and `orgdiv1` at USP in `pt_BR`

For running this notebook you'll need some external packages,
which can be installed with:

```shell
pip install bs4 fuzzywuzzy lxml pandas Python-Levenshtein unidecode
```

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 400 # Avoid "..." in large strings
pd.options.display.max_rows = 120     # Avoid "..." in the table representation of lengthy dataframes

## The `institution_orgdiv1` in the CSV created from raw XML documents

This is the Clea's CSV generated from raw XML documents:

In [2]:
dataset = pd.read_csv("inner_join_2018-06-04.csv",
                      dtype=str,
                      keep_default_na=False) \
            .drop_duplicates()

The goal isn't to normalize everything, but a subset of the data.
We're concerned with these columns/fields:

In [3]:
fields = [
    "addr_city",
    "addr_country",
    "addr_country_code",
    "addr_state",
    "institution_orgname",
    "institution_orgdiv1",
]
idataset = dataset[fields] # Input dataset

In [4]:
pd.DataFrame({
    "filled_in": (idataset.applymap(len) > 0).sum(),
    "distinct": idataset.apply(lambda col: col[col.apply(len) > 0].unique().size),
})

,filled_in,distinct
addr_city,79922,2728
addr_country,85968,266
addr_country_code,67724,124
addr_state,68545,671
institution_orgname,88527,9651
institution_orgdiv1,56518,9903


Which are the most common rows?

In [5]:
idataset.groupby(fields) \
        .size() \
        .rename("count") \
        .sort_values(ascending=False) \
        .head(10) \
        .reset_index()

,addr_city,addr_country,addr_country_code,addr_state,institution_orgname,institution_orgdiv1,count
0,,,,,,,537
1,São Paulo,Brazil,BR,SP,Universidade de São Paulo,Faculdade de Medicina,515
2,Santa Maria,Brazil,BR,RS,Universidade Federal de Santa Maria,,250
3,Belo Horizonte,Brazil,BR,MG,Universidade Federal de Minas Gerais,,234
4,Porto Alegre,Brazil,BR,RS,Universidade Federal do Rio Grande do Sul,,205
5,Santa Maria,Brazil,BR,RS,Universidade Federal de Santa Maria,Centro de Ciências Rurais,200
6,,Brasil,BR,,USP,Escola de Enfermagem,182
7,São Paulo,Brazil,BR,SP,Universidade de São Paulo,,169
8,São Paulo,Brazil,BR,SP,Universidade Federal de São Paulo,Escola Paulista de Medicina,164
9,São Paulo,Brazil,BR,SP,Universidade Federal de São Paulo,,155


USP (University of São Paulo) has $3$ entries in this top $10$.
There's no known number of typos, gross errors or inconsistencies for these fields,
but we can assume that any information
regarding normalizing this university name and its divisions' names
should be helpful.
On the other hand,
seeing the raw "first division" names that follows,
we know that:

* There are more than a single writing for the same school/college/institute;
* Sometimes their acronyms are used instead of their names;
* Usually they're in Brazilian Portuguese but sometimes they're not;
* Sometimes they're not school/college/institute names, but:
  * Internal departments names;
  * NAP (research group) names;
  * Graduate program names.

In [6]:
idataset[idataset["institution_orgname"].isin(["Universidade de São Paulo", "USP"])] \
        ["institution_orgdiv1"] \
        .drop_duplicates()

50                                                                             Faculdade de Odontologia
267                                                                             Instituto Oceanográfico
348                                                                           Instituto de Oceanografia
390                                                                                                    
941                                                                                  Escola Politécnica
988                                                                  Escola de Engenharia de São Carlos
1134                                                     Escola Superior de Agricultura Luiz de Queiroz
1194                                                                           Departamento de Ecologia
1214                                                   Escola Superior de Agricultura “Luiz de Queiroz”
1283                                                            

## Getting the name of institutes, colleges and schools at USP (University of São Paulo)

The Brazilian Portuguese names
for the USP institutes/colleges/schools
can be found
[here](http://www5.usp.br/institucional/escolas-faculdades-e-institutos/),
and the English names can be found
[here](http://www5.usp.br/english/institutional/escolas-faculdades-e-institutos-2/?lang=en).

In [7]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import etree

We can download the Brazilian Portuguese HTML:

In [8]:
pt_br_usp_ordgiv1_url = "http://www5.usp.br/institucional/escolas-faculdades-e-institutos/"
raw_html = urlopen(pt_br_usp_ordgiv1_url).read()
bsoup = BeautifulSoup(raw_html, "lxml")

Scraping the links with CSS is quite straightforward:

In [9]:
usp_orgdiv1_bsoup_info = pd.DataFrame([(el.text, el.attrs["href"])
                                       for el in bsoup.select(".post_content li a")],
                                       columns=["name", "url"])
usp_orgdiv1_bsoup_info.head()

,name,url
0,"Escola de Artes, Ciências e Humanidades (EACH)",http://each.uspnet.usp.br/
1,Escola de Comunicações e Artes (ECA),http://www.eca.usp.br/
2,Escola de Educação Física e Esporte (EEFE),http://www.eefe.usp.br/
3,Escola de Enfermagem (EE),http://www.ee.usp.br/
4,Escola Politécnica (Poli),http://www.poli.usp.br/


The cities aren't in these links,
but in the previous `<h2>` heading,
so we need to get them as well,
but bs4 loses the elements' order when using the
`.post_content li a | .post_content h2` CSS selector,
so we need to use something else.

With `lxml` directly
and a custom dictionary to match the campus city
with the school/college/institute element index, we get:

In [10]:
tree = etree.fromstring(raw_html, parser=etree.HTMLParser())
prefix_cutter_regex = re.compile("(?:Campus de |Em )(.*)")
a_h2_els = tree.xpath("//*[contains(@class, 'post_content')]//li/a | "
                      "//*[contains(@class, 'post_content')]//h2")

class PrevDict(dict):
    def __missing__(self, key):
        return next(self[k] for k in sorted(self, reverse=True) if k < key)

usp_campi = PrevDict((idx, prefix_cutter_regex.match(el.findtext("*")).groups()[0])
                     for idx, el in enumerate(a_h2_els) if el.tag == "h2")
usp_campi

{0: 'São Paulo',
 30: 'Bauru',
 32: 'São Carlos',
 38: 'Lorena',
 40: 'Piracicaba',
 43: 'Pirassununga',
 45: 'Ribeirão Preto',
 54: 'São Sebastião'}

In [11]:
usp_orgdiv1 = pd.DataFrame([(usp_campi[idx], el.text, el.attrib["href"])
                            for idx, el in enumerate(a_h2_els)
                            if el.tag == "a"],
                           columns=["city", "name", "url"])
usp_orgdiv1.tail()

,city,name,url
43,Ribeirão Preto,"Faculdade de Economia, Administração e Contabilidade de Ribeirão Preto (FEARP)",http://www.fearp.usp.br/
44,Ribeirão Preto,"Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto (FFCLRP)",http://www.ffclrp.usp.br/
45,Ribeirão Preto,Faculdade de Medicina de Ribeirão Preto (FMRP),http://www.fmrp.usp.br/
46,Ribeirão Preto,Faculdade de Odontologia de Ribeirão Preto (FORP),http://www.forp.usp.br/
47,São Sebastião,Centro de Biologia Marinha (CEBIMar),http://www.usp.br/cbm


Here in this notebook, these scraped names will be seen as
valid, normalized and consistent names for `addr_city` and `institution_orgdiv1`.

In [12]:
ndataset = usp_orgdiv1 \
    .rename(columns={"name": "institution_orgdiv1",
                     "city": "addr_city"}) \
    .assign(
        addr_country="Brazil",
        addr_country_code="BR",
        addr_state="SP",
        institution_orgname="Universidade de São Paulo (USP)",
    ).reindex(fields, axis=1)
ndataset # Ground-truth normalized dataset

,addr_city,addr_country,addr_country_code,addr_state,institution_orgname,institution_orgdiv1
0,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),"Escola de Artes, Ciências e Humanidades (EACH)"
1,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Escola de Comunicações e Artes (ECA)
2,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Escola de Educação Física e Esporte (EEFE)
3,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Escola de Enfermagem (EE)
4,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Escola Politécnica (Poli)
5,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Faculdade de Arquitetura e Urbanismo (FAU)
6,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Faculdade de Ciências Farmacêuticas (FCF)
7,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Faculdade de Direito (FD)
8,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),"Faculdade de Economia, Administração e Contabilidade (FEA)"
9,São Paulo,Brazil,BR,SP,Universidade de São Paulo (USP),Faculdade de Educação (FE)


Rows that have the same information from this normalized dataset,
perhaps with some missing field,
are consistent.

This table misses historical schools/colleges/institutes changes
and perhaps some nuances,
e.g. the music department at FFCLRP used to be the
[CMU-RP department at ECA](https://web.archive.org/web/20070614064907/http://www.musica.pcarp.usp.br/)
(i.e., it used to be a department of a school that happens to be in another city,
 $315 km$ away from the remaining departments).

## Fuzzy similarity (Levenshtein distance) per institution field with `fuzzywuzzy`

In [13]:
from fuzzywuzzy import process
import operator
from unidecode import unidecode

At first, let's perform a simple pre-normalization on the data
so that we don't have to worry about
the letter case and surrounding symbols.

In [14]:
def pre_normalize(name):
    return " ".join(re.sub("[^a-zA-Z ]", "", unidecode(name)).split()).lower()

In [15]:
npre = ndataset.applymap(pre_normalize)
ipre = idataset.applymap(pre_normalize)

In [16]:
norgnames = npre["institution_orgname"].unique()
norgnames

array(['universidade de sao paulo usp'], dtype=object)

In [17]:
iorgnames = ipre["institution_orgname"].unique()
iorgnames

array(['universidade federal do rio grande do sul ufrgs',
       'instituto federal sulriograndense ifsul',
       'universidade federal do rio de janeiro ufrj', ...,
       'universidade de lille i',
       'universidade federal de mato grosso de sul',
       'centro latinoamericano em sexualidade e direitos humanos'],
      dtype=object)

Using [`fuzzywuzzy`](https://github.com/seatgeek/fuzzywuzzy),
we can find the university names that best match the full USP name:

In [18]:
sorted(
    process.extractWithoutOrder(
        query=norgnames[0],
        choices=iorgnames,
        score_cutoff=88,
    ),
    key=operator.itemgetter(1),
    reverse=True,
)

[('universidade de sao paulo usp', 100),
 ('universidade de sao paulousp', 98),
 ('universidade de sao paulo fmusp', 97),
 ('universidade de sao paulo unesp', 97),
 ('universidade de sao paulo', 95),
 ('universidade sao paulo', 95),
 ('universidade de sao usp', 95),
 ('universidade de sao paulo sao paulo', 95),
 ('universidade de sao paulo hcfmusp', 94),
 ('universidade de sao paulo esalqusp', 92),
 ('univesidade de sao paulo', 91),
 ('universidade de sao paul', 91),
 ('universidad de sao paulo', 91),
 ('universidade de sao paulo ib', 91),
 ('universi dade de sao paulo', 91),
 ('universiade de sao paulo', 91),
 ('universidade de s paulo', 90),
 ('department of psychiatry and institute of psychiatry universidade de sao paulo usp',
  90),
 ('universidade de sao paulo brasil', 89),
 ('universidade federal de sao paulo', 88),
 ('universidade federal de sao paulo unifesp', 88),
 ('universidade estadual de sao paulo', 88),
 ('universidade cidade de sao paulo', 88),
 ('universidade anhanguera

Similarities in fuzzywuzzy range from $0$ (no common character)
to $100$ (equalness), using whole numbers.
In this example, similarities greater than $88$
are valid names for "Universidade de São Paulo (USP)".
However, this threshold is specific to this dataset,
and it doesn't seem something easy to find.

However, that's still brittle: if we remove the acronym suffix,
we get several false positives with similarity equal to $95$:

In [19]:
sorted(
    process.extractWithoutOrder(
        query="universidade de sao paulo",
        choices=iorgnames,
        score_cutoff=94,
    ),
    key=operator.itemgetter(1),
    reverse=True,
)

[('universidade de sao paulo', 100),
 ('univesidade de sao paulo', 98),
 ('universidade de sao paul', 98),
 ('universidad de sao paulo', 98),
 ('universi dade de sao paulo', 98),
 ('universiade de sao paulo', 98),
 ('universidade de s paulo', 96),
 ('universidade federal de sao paulo', 95),
 ('universidade sao paulo', 95),
 ('universidade de sao paulo usp', 95),
 ('universidade de sao paulo fmusp', 95),
 ('universidade estadual de sao paulo', 95),
 ('universidade de sao paulo hcfmusp', 95),
 ('universidade cidade de sao paulo', 95),
 ('universidade anhanguera de sao paulo', 95),
 ('universidade catolica de sao paulo', 95),
 ('universidade do estado de sao paulo', 95),
 ('universidade de sao paulo brasil', 95),
 ('universidade metodista de sao paulo', 95),
 ('universidade de sao paulo esalqusp', 95),
 ('universidade adventista de sao paulo', 95),
 ('universidade de sao paulo sao paulo', 95),
 ('universidade de sao paulo ib', 95),
 ('universidade de sao paulo unesp', 95),
 ('universidade

Another approach would be a comparison with all alternative names,
choosing the best one (i.e., reversing the input parameters),
but there's no other alternative name yet,
at least not for the university name.

Several university names have more information,
like the `ib`, `hcfmusp` and `esalqusp` suffix,
which might be useful for filling the `orgdiv1`.

Using $89$ as the threshold in the previous list
based on the full university name with its acronym,
and including the `"usp"` entry, we get these names:

In [20]:
usp_names = [name for name, score in process.extractWithoutOrder(
    query=norgnames[0],
    choices=iorgnames,
    score_cutoff=89,
)] + ["usp"]
usp_ipre_selector = ipre["institution_orgname"].isin(usp_names)
usp_names_raw = idataset[usp_ipre_selector]["institution_orgname"].unique().tolist()
usp_names_raw

['Universidade de São Paulo',
 'Universidade São Paulo',
 'USP',
 'Universidade de São Paulo (USP)',
 'Universidade de São Paulo (FMUSP)',
 'Universidade de São Paulo (HCFMUSP)',
 'Universidade de São Paulo - USP',
 'Universidade de Sao Paulo',
 'Universidade de São Paulo, Brasil (1976)',
 'Univesidade de Sao Paulo',
 'Universidade de São Paulo – USP',
 'Universidade de São – USP',
 'Universidade de S. Paulo',
 'Department of Psychiatry and Institute of Psychiatry, Universidade de São Paulo (USP)',
 'Universidade de São Paulo (ESALQ-USP)',
 'Universidade de São Paulo,',
 'Universidade de São Paul',
 'Universidad de São Paulo',
 '>Universidade de São Paulo',
 'U.S.P',
 'Universidade de São Paulo São Paulo',
 'Universidade de São Paulo IB',
 'Universi dade de São Paulo',
 'Universidade de São Paulo-USP',
 'Universidade de São Paulo/USP',
 'Universidade de São Paulo - FMUSP',
 'Universidade de São Paulo, UNESP',
 'Universiade de São Paulo',
 '), Universidade de São Paulo',
 'Universidade 

There are $409$ distinct `orgdiv1` names
in $6479$ entries for this university.

In [21]:
usp_ipre = ipre[usp_ipre_selector]
usp_ipre.shape

(6479, 6)

In [22]:
d1inames = usp_ipre["institution_orgdiv1"].drop_duplicates()
d1inames.shape

(409,)

Let's find the best match for all of them, using fuzzywuzzy:

In [23]:
norgdiv1s = npre["institution_orgdiv1"].unique()
od1_normalization_table = pd.DataFrame(
    d1inames.apply(
        lambda name:
            name
                and process.extractOne(query=name, choices=norgdiv1s, score_cutoff=80)
                or ("", 0)
    ).values.tolist(),
    index=d1inames.index,
    columns=["normalized", "similarity"],
).assign(orgdiv1=d1inames) \
 .reindex(["orgdiv1", "normalized", "similarity"], axis=1) \
 .sort_values(
     by=["similarity", "normalized", "orgdiv1"],
     ascending=[False, True, True],
 )
od1_normalization_table[lambda df: df["similarity"] >= 90]

,orgdiv1,normalized,similarity
29887,escola de artes ciencias e humanidades each,escola de artes ciencias e humanidades each,100
2445,escola de engenharia de sao carlos eesc,escola de engenharia de sao carlos eesc,100
1134,escola superior de agricultura luiz de queiroz,escola superior de agricultura luiz de queiroz,100
4026,faculdade de medicina de ribeirao preto fmrp,faculdade de medicina de ribeirao preto fmrp,100
77496,faculdade de medicina veterinaria e zootecnia fmvz,faculdade de medicina veterinaria e zootecnia fmvz,100
87370,instituto de ciencias biomedicas icb,instituto de ciencias biomedicas icb,100
86258,escola superior de agricultutra luiz de queiroz,escola superior de agricultura luiz de queiroz,99
85799,escola superior de agricultura luis de queiroz,escola superior de agricultura luiz de queiroz,98
6208,instituto de astronomia geofisica e ciencias atmosfericas,instituto de astronomia geofisica e ciencias atmosfericas iag,97
35600,faculdade de economia administracao e contabilidade de ribeirao preto,faculdade de economia administracao e contabilidade de ribeirao preto fearp,96


In [24]:
od1_normalization_table[lambda df: df["similarity"] >= 90].shape

(89, 3)

These are the pre-normalized names.
Let's see this fuzzywuzzy-based normalization
applied to the actual `orgdiv1` values
as they appear in the XML files:

In [25]:
npre_od1_pairs = ndataset[["institution_orgdiv1"]].assign(nn=npre["institution_orgdiv1"])
ipre_od1_pairs = idataset[usp_ipre_selector].assign(ni=ipre["institution_orgdiv1"])
fw_result = pd.merge(
    ipre_od1_pairs,
    pd.merge(od1_normalization_table, npre_od1_pairs, left_on="normalized", right_on="nn")
      .rename({"institution_orgdiv1": "institution_orgdiv1_normalized"}, axis=1),
    left_on="ni",
    right_on="orgdiv1",
).drop(columns=["ni", "orgdiv1", "normalized", "nn"]) \
 .rename({"institution_orgdiv1": "orgdiv1",
          "institution_orgdiv1_normalized": "normalized"},
         axis=1) \
 .groupby(["similarity", "normalized", "orgdiv1"]) \
 .size() \
 .rename("count") \
 .reset_index() \
 .reindex(["count", "orgdiv1", "normalized", "similarity"], axis=1) \
 .sort_values(
     by=["similarity", "normalized", "orgdiv1"],
     ascending=[False, True, True],
 )
fw_result[lambda df: df["similarity"] >= 90]

,count,orgdiv1,normalized,similarity
320,27,"Escola Superior de Agricultura ""Luiz de Queiroz""",Escola Superior de Agricultura “Luiz de Queiroz”,100
321,1,Escola Superior de Agricultura 'Luiz de Queiroz',Escola Superior de Agricultura “Luiz de Queiroz”,100
322,58,Escola Superior de Agricultura Luiz de Queiroz,Escola Superior de Agricultura “Luiz de Queiroz”,100
323,3,Escola Superior de Agricultura Luiz de Queiróz,Escola Superior de Agricultura “Luiz de Queiroz”,100
324,1,"Escola Superior de Agricultura “Luiz de Queiroz""",Escola Superior de Agricultura “Luiz de Queiroz”,100
325,44,Escola Superior de Agricultura “Luiz de Queiroz”,Escola Superior de Agricultura “Luiz de Queiroz”,100
326,2,Escola Superior de Agricultura “Luiz de Queiróz”,Escola Superior de Agricultura “Luiz de Queiroz”,100
327,1,"Escola de Artes, Ciências e Humanidades (EACH)","Escola de Artes, Ciências e Humanidades (EACH)",100
328,2,Escola de Engenharia de São Carlos (EESC),Escola de Engenharia de São Carlos (EESC),100
329,1,Escola de Engenharia de São Carlos – EESC,Escola de Engenharia de São Carlos (EESC),100


In [26]:
# Number of normalizations performed
fw_result[lambda df: df["similarity"] >= 90]["count"].sum()

4010

Only $89$ distinct pre-normalized values out of $409$
had a similarity of at least $90$,
but these are $4010$ entries in this dataset,
out of $6479$, that's more than $60\%$ of the data.

There are a lot of entries without the acronym,
but that doesn't seem to be an issue:
the default fuzzywuzzy comparison scorer is breaking the input as words
(else these inputs with several extra/distinct words wouldn't match).

Let's see the remaining normalization entries
that appeared at least $4$ times.

In [27]:
fw_result[lambda df: (df["similarity"] < 90) & (df["count"] > 3)]

,count,orgdiv1,normalized,similarity
214,5,"Hospital das Clínicas, Faculdade de Medicina de Ribeirão Preto",Faculdade de Medicina de Ribeirão Preto (FMRP),89
208,4,Faculdade de Medicina de São Paulo,Faculdade de Medicina (FM),88
12,8,Departamento de Cirurgia,"Escola de Artes, Ciências e Humanidades (EACH)",86
14,5,Departamento de Economia,"Escola de Artes, Ciências e Humanidades (EACH)",86
19,4,Departamento de Genética,"Escola de Artes, Ciências e Humanidades (EACH)",86
22,6,Departamento de Imunologia,"Escola de Artes, Ciências e Humanidades (EACH)",86
23,7,Departamento de Neurologia,"Escola de Artes, Ciências e Humanidades (EACH)",86
24,4,Departamento de Oftalmologia,"Escola de Artes, Ciências e Humanidades (EACH)",86
25,6,Departamento de Patologia,"Escola de Artes, Ciências e Humanidades (EACH)",86
28,4,Departamento de Psiquiatria,"Escola de Artes, Ciências e Humanidades (EACH)",86


When the similarity gets below $90$, it fails.
As it seems, $86$ is a rather bad result,
these happened because a single word had a full match,
like *Instituto* in the latest entries,
the *de* preposition in most entries,
cities like *Bauru* and *Ribeirão Preto*, etc..
Therefore, our threshould value should always be greater than $86$.